# Export data to files with Suitcase

In this notebook you will:

* Export saved data to CSV and TIFF.
* See a preview of how exporting data live during a scan works.

## Configuration
Below, we will connect to EPICS IOC(s) controlling simulated hardware in lieu of actual motors, detectors. The IOCs should already be running in the background. Run this command to verify that they are running: it should produce output with RUNNING on each line. In the event of a problem, edit this command to replace `status` with `restart all` and run again.

In [1]:
!supervisorctl -c supervisor/supervisord.conf status

decay                            RUNNING   pid 4902, uptime 0:00:33
mini_beamline                    RUNNING   pid 4903, uptime 0:00:33
random_walk                      RUNNING   pid 4904, uptime 0:00:33
random_walk_horiz                RUNNING   pid 4918, uptime 0:00:30
random_walk_vert                 RUNNING   pid 4906, uptime 0:00:33
simple                           RUNNING   pid 4907, uptime 0:00:33
thermo_sim                       RUNNING   pid 4908, uptime 0:00:33
trigger_with_pc                  RUNNING   pid 4909, uptime 0:00:33


In [2]:
%run scripts/beamline_configuration.py

## Scalar data to CSV

In [3]:
RE(scan([ph], motor_ph, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 3     Time: 2020-05-19 16:38:44
Persistent Unique Scan ID: '13a0e3b2-491b-480c-a3b2-5473c1cfc5cf'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+------------+
|   seq_num |       time |   motor_ph |     ph_det |
+-----------+------------+------------+------------+
|         1 | 16:38:44.4 |     -1.000 |     102107 |


|         2 | 16:38:44.4 |     -0.500 |     103240 |
|         3 | 16:38:44.5 |      0.000 |     103987 |
|         4 | 16:38:44.5 |      0.500 |     103381 |
|         5 | 16:38:44.5 |      1.000 |     101694 |
+-----------+------------+------------+------------+


generator scan ['13a0e3b2'] (scan num: 3)





('13a0e3b2-491b-480c-a3b2-5473c1cfc5cf',)

In [4]:
import suitcase.csv

In [5]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/13a0e3b2-491b-480c-a3b2-5473c1cfc5cf-primary.csv')]}

In [6]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}-{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni-12345-primary.csv')]}

In [7]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary.csv')]}

## Image data to TIFF

In [8]:
RE(scan([spot], mtr_spotx, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 4     Time: 2020-05-19 16:38:44
Persistent Unique Scan ID: 'c77adaf2-8b44-474a-af31-5b0a66c9d87b'


New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+-------------+------------+
|   seq_num |       time | motor_spotx |   spot_roi |
+-----------+------------+-------------+------------+
|         1 | 16:38:44.9 |          -1 | 1106124594 |


|         2 | 16:38:45.1 |          -0 | 1099434970 |


|         3 | 16:38:45.2 |           0 | 1087261783 |
|         4 | 16:38:45.3 |           0 | 1080500440 |


|         5 | 16:38:45.4 |           1 | 1074467789 |


+-----------+------------+-------------+------------+
generator scan ['c77adaf2'] (scan num: 4)





('c77adaf2-8b44-474a-af31-5b0a66c9d87b',)

### TIFF stack, a 3D block of images in one file

In [9]:
import suitcase.tiff_stack

In [10]:
suitcase.tiff_stack.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary-spot_img.tiff')]}

### TIFF series

In [11]:
import suitcase.tiff_series

Unlike CSV or TIFF stack, this exporter creates separate file per row (Event document). We can include information from a specific Event in the filename. Here, we include the motor position.

In [12]:
suitcase.tiff_series.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}/{event[data][motor_spotx]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-1.0-primary-spot_img-0.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-0.5-primary-spot_img-1.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.0-primary-spot_img-2.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.5-primary-spot_img-3.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/1.0-primary-spot_img-4.tiff')]}

## "Beta" feature: Set up live exporting

*We plan to streamline this in the future, but here is a demonstration that it is possible.*

In [13]:
# Execute this once to configure RE to send data to an exporter.
# At the beamline, this would normally be done in a script automatically at startup.

from event_model import RunRouter
from databroker.core import discover_handlers

def factory(name, start_doc):

    serializer = suitcase.tiff_series.Serializer(
        'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-{start[uid]:.8}-')

    def cb(name, doc):
        serializer(name, doc)

    return [cb], []


rr = RunRouter([factory], discover_handlers())
RE.subscribe(rr)

2

In [14]:
RE(count([spot], num=30, delay=1), sample_comp='Lab6', barcode='12345')



Transient Scan ID: 5     Time: 2020-05-19 16:38:45
Persistent Unique Scan ID: '0e7830e6-9c15-4cbd-8090-9b5012f7abb4'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+
|   seq_num |       time |   spot_roi |
+-----------+------------+------------+
|         1 | 16:38:45.9 | 1057396102 |


|         2 | 16:38:46.8 | 1075800744 |


|         3 | 16:38:47.8 | 1128177482 |


...

|        28 | 16:39:12.8 | 1106713689 |


|        29 | 16:39:13.8 | 1055829221 |


|        30 | 16:39:14.8 | 1078796081 |


+-----------+------------+------------+
generator count ['0e7830e6'] (scan num: 5)





('0e7830e6-9c15-4cbd-8090-9b5012f7abb4',)